In [ ]:
# Here is the time comparison
import JSSEnv
import src.jss_graph_env.disjunctive_graph_jss_env as jss_env
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances

import src.jsp_instance_parser 
import time
import gym
import pandas as pd
import random
import numpy as np
from src.jss_lite.jss_lite import jss_lite

index=['klagenfurt','alex','lite']
df=pd.DataFrame(np.zeros((3,5)),columns=['setuptime','resettime','50xactiontime','col4','col5'],index=index)
#df=pd.DataFrame
#df.set_index('index')
for i in range(100):

    def timer(innerfun):
        tmp_t=time.time()
        tmp_r=innerfun()
        return time.time()-tmp_t,tmp_r

    tmp_t=time.time()
    env2=gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/dmu79.txt'})
    t_1=(time.time()-tmp_t)
    df['setuptime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    instance_list=['resources/jsp_instances/standard/ft06.txt','resources/jsp_instances/standard/ft10.txt','resources/jsp_instances/standard/ft20.txt','resources/jsp_instances/standard/abz5.txt','resources/jsp_instances/standard/abz6.txt','resources/jsp_instances/standard/abz7.txt']
#instance_list=['resources/jsp_instances/standard/ft06.txt']
    env_3=jssp_light_obs_wrapper_multi_instances(instances_list=instance_list)
    #env_3=jss_lite(instance_path='resources/jsp_instances/standard/dmu79.txt')
    t_1=time.time()-tmp_t
    df['setuptime']['lite']+=t_1/100




    tmp_t=time.time()
    path='resources/jsp_instances/standard/dmu79.txt'
    curr_instance=src.jsp_instance_parser.parse_jps_standard_specification(path)
    res,std_matrix=curr_instance
    env_alex = jss_env.DisjunctiveGraphJssEnv(res,default_visualisations='gantt_console')
    t_2=(time.time()-tmp_t)
    df['setuptime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env2.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    t2=env_alex.reset()
    t_2=(time.time()-tmp_t)
    df['resettime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env_3.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['lite']+=t_1/100


    state=env2.reset()
    for i in range(50):
        legal_action=state['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env2.step(action)
        t_1=(time.time()-tmp_t)
        df['50xactiontime']['klagenfurt']+=t_1/50

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break
    state=env_3.reset()
    for i in range(50):
        legal_action=env_3.get_legal_actions()
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env_3.step(action)
        t_1=(time.time()-tmp_t)
        df['50xactiontime']['lite']+=t_1/50

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break



    action_list=np.arange(0,env_alex.n_jobs*env_alex.n_machines)
    for i in range(50):
        action_mask=env_alex.valid_action_mask()
        actions=action_list[action_mask]
        next_action=random.choice(actions)
        #next_action=actions[action_mask[0]]
        #print(next_action)
        tmp_t=time.time()
        state,reward,done,info =env_alex.step(next_action)
        t_2=(time.time()-tmp_t)
        df['50xactiontime']['alex']+=t_2/50
        if done == True:
            print(f"finished after {i} steps")
            break
        #env.render()


print(df)
